In [1]:
import psycopg2
from psycopg2 import sql
import datetime

In [2]:
hostName = "hish-db-01.cfwyts8tlkjs.us-east-1.rds.amazonaws.com"

In [3]:
engine = psycopg2.connect(
    database="postgres",
    user="masteruser",
    password="hishmaster123",
    host=hostName,
    port='5432'
)

In [4]:
cur = engine.cursor()

In [5]:
engine.autocommit = True

Create New User

In [7]:
def createNewUser(userName, userPassword):
    
    newTableName = f"{userName}Data"
    
    sqlQuery = sql.SQL("""
    CREATE ROLE {role} WITH LOGIN PASSWORD %s;
    GRANT CONNECT ON DATABASE postgres TO {role};

    CREATE TABLE {table} (
    patientID INT PRIMARY KEY,
    date TIMESTAMP NOT NULL,
    condition VARCHAR ( 50 ) NOT NULL
    );

    GRANT SELECT, INSERT, UPDATE, DELETE ON TABLE {table} TO {role};
    """).format(
            role=sql.Identifier(userName),
            table=sql.Identifier(newTableName),
    )
    cur.execute(sqlQuery, (userPassword,))
    

In [7]:
newUserName1 = "user1"
newUserPassword1 = "t123456"
createNewUser(newUserName, newUserPassword)

In [8]:
newUserName3 = "user3"
newUserPassword3 = "t123456"
createNewUser(newUserName3, newUserPassword3)

In [9]:
engine.close()

Sign in and view table

In [10]:
def userSignIn(userName, userPassword):
    user_engine = psycopg2.connect(
        database="postgres",
        user=userName,
        password=userPassword,
        host=hostName,
        port='5432'
    )
    
    user_cur = user_engine.cursor()
    user_engine.autocommit = True
    
    userTableName = f"{userName}Data"
    user_cur.execute(sql.SQL("SELECT * FROM {table}").format(table=sql.Identifier(userTableName)))
    print(user_cur.fetchall())
    
    return user_engine, user_cur

In [11]:
newUserName1 = "user1"
newUserPassword1 = "t123456"
user_signin_engine, user_signin_cur = userSignIn(newUserName1, newUserPassword1)

[(1, datetime.datetime(2023, 3, 17, 0, 0), 'Negative')]


In [12]:
user_signin_engine, user_signin_cur = userSignIn(newUserName3, newUserPassword3)

[]


In [13]:
user_signin_engine.close()

Insert table and view update

In [14]:
def userAddData(userName, userPassword, newPatientID, newPatientDate, newPatientCondition):
    user_engine = psycopg2.connect(
        database="postgres",
        user=userName,
        password=userPassword,
        host=hostName,
        port='5432'
    )
    
    user_cur = user_engine.cursor()
    user_engine.autocommit = True
    
    userTableName = f"{userName}Data"
    
    sqlQuery = sql.SQL("""
    INSERT INTO {table} VALUES (%s, %s, %s); 
    SELECT * FROM {table}
    """).format(table=sql.Identifier(userTableName))
    
    user_cur.execute(sqlQuery, (newPatientID, newPatientDate, newPatientCondition))
    print(user_cur.fetchall())
    
    return user_engine, user_cur

In [15]:
user_insert_engine, user_insert_cur = userAddData(newUserName3, newUserPassword3, 3, datetime.date(2023, 3, 17), "Negative")

[(3, datetime.datetime(2023, 3, 17, 0, 0), 'Negative')]


In [16]:
user_insert_engine.close()

In [17]:
user_signin_engine, user_signin_cur = userSignIn(newUserName3, newUserPassword3)

[(3, datetime.datetime(2023, 3, 17, 0, 0), 'Negative')]


In [18]:
user_signin_engine.close()

In [19]:
user_signin_engine, user_signin_cur = userSignIn(newUserName1, newUserPassword1)

[(1, datetime.datetime(2023, 3, 17, 0, 0), 'Negative')]


In [20]:
user_signin_engine.close()